In [ ]:
# first download Neo4j desktop and create a database of version 5.11 and install the plugins: Neosemantics, APOC

In [ ]:
!pip install -U sentence-transformers rank_bm25
!pip install stop_words
!nvidia-smi

  Obtaining dependency information for torch>=1.11.0 from https://files.pythonhosted.org/packages/96/23/18b9c16c18a77755e7f15173821c7100f11e6b3b7717bea8d729bdeb92c0/torch-2.2.2-cp311-none-macosx_11_0_arm64.whl.metadata
  Using cached torch-2.2.2-cp311-none-macosx_11_0_arm64.whl.metadata (25 kB)
Using cached torch-2.2.2-cp311-none-macosx_11_0_arm64.whl (59.7 MB)

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
zsh:1: command not found: nvidia-smi


In [ ]:
# Importing required libraries
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch

In [ ]:
# For semantic search, we use SentenceTransformer('multi-qa-MiniLM-L6-cos-v1') to encode all passages.

bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256
top_k = 100

In [ ]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


In [ ]:
# this is the context information for using semantic search
passages = ["Environmental sustainability is the ability to maintain an ecological balance in our planet's natural environment and conserve natural resources to support the wellbeing of current and future generations.",
            "Climate action: Acting now to stop global warming. Life below water: Avoiding the use of plastic bags to keep the oceans clean. Life on land: Planting trees to help protect the environment. Responsible consumption and production: Recycling items such as paper, plastic, glass and aluminum.",
            "Characteristics of sustainability or sustainable development are: Reduce emission of greenhouse gases, which will reduce global warming and help in preserving the environment. Use of natural and biodegradable materials for reducing the impact on the environment.",
            "However, it refers to four distinct areas: environmental, social, economic, and human   – known as the four pillars of sustainability.",
            "Environmental sustainability is important because of how much energy, food, and human-made resources we use every day. Rapid population growth has resulted in increased farming and manufacturing, leading to more greenhouse gas emissions, unsustainable energy use, and deforestation.",
            "Sustainability maintains the health and biocapacity of the environment. Sustainability supports the well-being of individuals and communities. Sustainability promotes a better economy where there is little waste and pollution, fewer emissions, more jobs, and a better distribution of wealth."]

print("Number of Passages:", len(passages))

Passages: 6


In [ ]:
corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from rank_bm25 import BM25Okapi
#from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction import _stop_words
# from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np

# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
  tokenized_doc = []
  for token in text.lower().split():
    token = token.strip(string.punctuation)

    if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
      tokenized_doc.append(token)
  return tokenized_doc

tokenized_corpus = []
for passage in tqdm(passages):
  tokenized_corpus.append(bm25_tokenizer(passage))

print("tokenized_corpus...", tokenized_corpus)
# First thing to do is create an instance of the BM25 class, which reads in a corpus of text and does some indexing on it.
bm25 = BM25Okapi(tokenized_corpus)

  0%|          | 0/6 [00:00<?, ?it/s]

tokenized_corpus... [['environmental', 'sustainability', 'ability', 'maintain', 'ecological', 'balance', "planet's", 'natural', 'environment', 'conserve', 'natural', 'resources', 'support', 'wellbeing', 'current', 'future', 'generations'], ['climate', 'action', 'acting', 'stop', 'global', 'warming', 'life', 'water', 'avoiding', 'use', 'plastic', 'bags', 'oceans', 'clean', 'life', 'land', 'planting', 'trees', 'help', 'protect', 'environment', 'responsible', 'consumption', 'production', 'recycling', 'items', 'paper', 'plastic', 'glass', 'aluminum'], ['characteristics', 'sustainability', 'sustainable', 'development', 'reduce', 'emission', 'greenhouse', 'gases', 'reduce', 'global', 'warming', 'help', 'preserving', 'environment', 'use', 'natural', 'biodegradable', 'materials', 'reducing', 'impact', 'environment'], ['refers', 'distinct', 'areas', 'environmental', 'social', 'economic', 'human', '–', 'known', 'pillars', 'sustainability'], ['environmental', 'sustainability', 'important', 'energ

In [ ]:

# query is the input given by the user which need to be searched in the above 6 passages through keyword and semantic search.



def search(query):
  print("Input question:", query)

  ####### Keyword Search through BM25 algorithm ########
  bm25_scores = bm25.get_scores(bm25_tokenizer(query))
  top_n = np.argpartition(bm25_scores, -5)[-5:]
  bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
  bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
  #print("Top-5 lexical search (BM25) hits")
  #for hit in bm25_hits[0:5]:
      #print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

  ##### Sematic Search #####
  torch.set_default_device("mps")
  question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
  question_embedding = question_embedding.to("mps")
  # Both question and corpus are encoded
  hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
  hits = hits[0]

  # First, we use a Bi-Encoder to retrieve a list of result candidates,
  # then you use a Cross-Encoder on this list of candidates to pick out (or rerank) the most relevant results.
  # This way, we benefit from the efficient retrieval method using Bi-Encoders
  # and the high accuracy of the Cross-Encoder, so we can use this on large scale datasets!

  ##### Re-Ranking #####
  # Now, score all retrieved passages with the cross_encoder
  cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
  cross_scores = cross_encoder.predict(cross_inp)

  # Sort results by the cross-encoder scores
  for idx in range(len(cross_scores)):
      hits[idx]['cross-score'] = cross_scores[idx]


  #### Output of top-5 cases ######
  #print("Top-5 Bi-Encoder Retrieval hits")
  hits = sorted(hits, key=lambda x: x['score'], reverse=True)
  #for hit in hits[0:5]:
      #print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

  print("Top-5 Cross-Encoder Re-ranker hits")
  hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
  hit = hits[0]
  #for hit in hits[0:5]:
  #print("\t{:.3f}\t{}".format(hit['cross-score'], passages[hit['corpus_id']].replace("\n", " ")))
  print(passages[hit['corpus_id']])
  return passages[hit['corpus_id']]

fetched = search(query = "What is the second domain of sustainability?")
fetched

Input question: What is the second domain of sustainability?
Top-5 Cross-Encoder Re-ranker hits
However, it refers to four distinct areas: environmental, social, economic, and human   – known as the four pillars of sustainability.


'However, it refers to four distinct areas: environmental, social, economic, and human   – known as the four pillars of sustainability.'

In [ ]:

# if the context information are only strings
from langchain_community.llms import Replicate

os.environ["REPLICATE_API_TOKEN"] = "ENTER REPLICATE API KEY HERE"

llm = Replicate(model="mistralai/mixtral-8x7b-instruct-v0.1:cf18decbf51c27fed6bbdc3492312c1c903222a56e3fe9ca02d6cbe5198afc10", model_kwargs={"temperature": 0.75, "max_length": 500, "top_p": 1},)
prompt = """
given the context here: {}, answer the question:
What is the second domain of sustainability?
""".format(fetched)

# it might take a while for replicate to boot the model in their hardware
llm(prompt)

In [ ]:
# if we want to store the context information into a vector database

from reportlab.pdfgen.canvas import Canvas
canvas = Canvas("GFG.pdf")
canvas.drawString(100, 100, fetched)
canvas.showPage()
canvas.save()


In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

### Putting the result of semantic search in the below pdf
pdf_loader = PyPDFLoader('GFG.pdf'  )
documents = pdf_loader.load()
document = list(documents[0])[0][1]
document

'However, it refers to four distinct areas: environmental, social, economic, and human   – known as the four pillars of sustainability.\n'

In [ ]:
# we split the data into chunks of 1,000 characters, with an overlap
# of 200 characters between the chunks, which helps to give better results
# and contain the context of the information between chunks

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)


# we create our vectorDB, using the OpenAIEmbeddings transformer to create
# embeddings from our text chunks. We set all the db information to be stored
# inside the ./data directory, so it doesn't clutter up our source files

vectordb = Chroma.from_documents(
  documents,
  embedding=OpenAIEmbeddings(),
  persist_directory='./data2'
)
# vectordb.persist()
vectordb


/Users/jasonliu/PycharmProjects/pythonProject/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain_community.llms import Replicate

os.environ["REPLICATE_API_TOKEN"] = "ENTER REPLICATE API KEY HERE"

qa_chain1 = RetrievalQA.from_chain_type(
    llm = Replicate(model="mistralai/mixtral-8x7b-instruct-v0.1:cf18decbf51c27fed6bbdc3492312c1c903222a56e3fe9ca02d6cbe5198afc10", model_kwargs={"temperature": 0.75, "max_length": 500, "top_p": 1},),
    retriever=vectordb.as_retriever(search_kwargs={'k': 7}),
    return_source_documents=True
)

In [ ]:
# we can now execute queries against our Q&A chain
result = qa_chain1({'query': 'What is the second domain of sustainability?'})
print(result['result'])

/Users/jasonliu/PycharmProjects/pythonProject/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Number of requested results 7 is greater than number of elements in index 2, updating n_results = 2


The second domain of sustainability, based on the "four pillars of sustainability" you mentioned, is social sustainability. This domain focuses on issues such as social equity, human well-being, and relationships between communities. It's about creating a society that is fair, inclusive, and provides opportunities for all its members to thrive.
